In [1]:
import pandas as pd
import datetime as dt

Counting the total rows in the csv file and loading only 75% into the pandas dataframe.

In [2]:
# Defining the file path
file_path = "mock_data.csv"  #using the name only because it is in the same directory as this notebook

# Getting total rows count minus the header
total_rows = sum(1 for _ in open(file_path)) - 1  # -1 to exclude the header

# Computing only 75% of total rows
rows_to_load = int(0.75 * total_rows)

# Loading only 75% of the dataset into the dataframe
df = pd.read_csv(file_path, nrows=rows_to_load)

# Displaying the first ten rows to confirm if it loaded correctly
df.head(10)



,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29.000000000
1,1029,China,47,2016-01-08 03:13:12.000000000
2,1025,Nigeria,10,2023-02-20 19:06:14.000000000
3,1006,United Kingdom,48,2022-05-26 12:46:35.000000000
4,1024,Philippines,16,2016-12-13 03:44:04.000000000
5,1033,Indonesia,20,2016-05-26 22:29:44.000000000
6,1017,Paraguay,14,2022-09-08 16:30:56.000000000
7,1035,Honduras,15,2021-04-14 18:55:03.000000000
8,1029,Argentina,14,2020-11-01 10:21:03.000000000
9,1014,China,3,2020-04-14 00:51:05.000000000


Checking the initial length of the dataframe for comparison after dropping duplicates
Removing consecutive duplicate rows
Checking the new length of dataframe after dropping duplicates

In [3]:
# Removing only consecutive duplicates and keeping the first as the original
df = df.drop_duplicates(keep="first")
len(df)

750

Converting sign up date to datetime format and also replacing dates from 2020 with NaN(using NaT because it is a datetime column)

In [4]:
# Converting "Sign-up Date" to datetime, error handling for inaccurate signup dates 
df["Sign-up Date"] = pd.to_datetime(df["Sign-up Date"], errors="coerce")

# Replacing dates from the year 2020 with NaN
df.loc[df["Sign-up Date"].dt.year == 2020, "Sign-up Date"] = pd.NaT


In [5]:
df.head(20)

,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29
1,1029,China,47,2016-01-08 03:13:12
2,1025,Nigeria,10,2023-02-20 19:06:14
3,1006,United Kingdom,48,2022-05-26 12:46:35
4,1024,Philippines,16,2016-12-13 03:44:04
5,1033,Indonesia,20,2016-05-26 22:29:44
6,1017,Paraguay,14,2022-09-08 16:30:56
7,1035,Honduras,15,2021-04-14 18:55:03
8,1029,Argentina,14,NaT
9,1014,China,3,NaT


Removing customers in canada with less than 5 purchases.Writing two codes for this and commenting the longer one out.

In [6]:
df = df.drop(df[(df["Total Purchases"] < 5) & (df["Country"] == "Canada")].index)
#filtered_df = df[(df["Total Purchases"] < 5) & (df["Country"] == "Canada")]
#print(filtered_df)
#rows_to_remove = filtered_df.index
#print(rows_to_remove)
#df = df.drop(rows_to_remove)




Creating a new column loyalty score, based on the given conditions

In [7]:

def assign_loyalty_score(row):
    if row["Total Purchases"] > 20:
        return 3 #assign 3 if total purchases greater than 20
    elif 10 <= row["Total Purchases"] <= 20:
        return 2 #assign 2 if total purchases greater than 10
    else:  # if total Purchases less than 10
       # if pd.notna(row["Sign-up Date"]):  # Ensure valid sign-up date
         #   membership_years = (pd.Timestamp.today() - row["Sign-up Date"]).days / 365 #calculating membership years
          #  if membership_years > 3:
          #      return 2
        #return 1
     if pd.notna(row["Sign-up Date"]) and (pd.Timestamp.today().year - row["Sign-up Date"].year) > 3:
            return 2
     else:
            return 1

df["Loyalty Score"] = df.apply(assign_loyalty_score, axis=1) #apply the function to each row in the dataframe


In [8]:
df#printing df to confirm
most_common_loyalty_score = df["Loyalty Score"].value_counts()
print(most_common_loyalty_score)


3    420
2    275
1     53
Name: Loyalty Score, dtype: int64


Grouping the data by country and calculating aggregrates for each country except Mexico

In [9]:
# New dataframe that excludes Mexico
df_filtered = df[df["Country"] != "Mexico"]

# Group by Country and compute statistics
country_stats = df_filtered.groupby("Country").agg(
    Avg_Purchases=("Total Purchases", "mean"),
    Max_Purchases=("Total Purchases", "max"),
    Most_Common_Loyalty_Score=("Loyalty Score", lambda x: x.mode()[0] if not x.mode().empty else None)
).reset_index()

# Display results
country_stats


,Country,Avg_Purchases,Max_Purchases,Most_Common_Loyalty_Score
0,Afghanistan,8.000000,20,2
1,Albania,20.500000,38,2
2,Andorra,11.000000,11,2
3,Angola,15.000000,29,2
4,Argentina,24.692308,45,3
...,...,...,...,...
106,Vanuatu,22.000000,22,3
107,Venezuela,25.333333,43,3
108,Vietnam,24.333333,47,3
109,Yemen,10.000000,15,2


Getting today's date in the specified format
Creating the output filename
Saving the processed data in a csv format


In [10]:
today_date = dt.datetime.today().strftime("%d-%m-%Y")
print(today_date)
output_filename = f"processed_data_{today_date}.csv"
df[["Customer ID", "Loyalty Score", "Country", "Total Purchases"]].to_csv(output_filename, index=False)
print(f"File saved as: {output_filename}")



21-03-2025
File saved as: processed_data_21-03-2025.csv


In [11]:
df.sample(10) #printing unsorted sample

,Customer ID,Country,Total Purchases,Sign-up Date,Loyalty Score
434,1013,Greece,18,2023-04-06 06:32:10,2
385,1008,Sweden,47,2015-08-29 19:25:25,3
84,1007,Indonesia,29,2023-11-25 17:46:32,3
525,1022,China,28,2018-12-28 01:00:26,3
739,1022,China,21,2021-09-19 09:36:55,3
208,1011,Vanuatu,22,2019-01-04 20:55:06,3
540,1029,Indonesia,23,2017-07-25 12:05:05,3
9,1014,China,3,NaT,1
503,1026,China,14,2019-07-04 11:12:42,2
354,1032,Czech Republic,41,2023-11-03 00:37:39,3


Data Processing Summary Report:
1.The most common Loyalty Score is 3, meaning most customers have made more than 20 purchases and the company has a great customer retention.
2.Mexico was excluded from the aggregation which may alter the results and lead to data quality issues.
3.Feature engineering made it easier to perform aggregations on the data
4.Cleaning the data has ensured that only meaningful and relevant customer data remains, improving the quality of the data.
5.The Loyalty Score distribution can help businesses identify and target high-value customers .